In [1]:
import requests
import base64
import json
import hashlib

import mysql.connector
from dotenv import load_dotenv
import os


In [12]:
def leeactzl(user, repo_name, path_to_file):
    json_url ='https://api.github.com/repos/{}/{}/contents/{}'.format(user, repo_name,
                                                                      path_to_file)
    response = requests.get(json_url) #get data from json file located at specified URL 

    if response.status_code == requests.codes.ok:
        jsonResponse = response.json()  # the response is a JSON
        #the JSON is encoded in base 64, hence decode it
        content = base64.b64decode(jsonResponse['content'])
        #convert the byte stream to string
        jsonString = content.decode('utf-8')
        return json.loads(jsonString)
 
    else:
        return 'Content was not found.'
    
def calcquerys(dt_query):
    
    regq = ''
    query = []

    for y in dt_query.keys():
        z = dt_query[y]
        asi = z['op'].lower()
        gna = instrucions[asi][0]
        fl  = instrucions[asi][1]
        regq += z['op'] +' ' + y + ' ' + gna + z['set'] + fl + z['filtro']
        query.append(z['op'] +' ' + y + ' ' + gna.upper() + z['set'] + fl.upper() + z['filtro'])

    return regq, query

def conexion(querys):
    
    cnx = mysql.connector.connect(user=sql_id, password=sql_pw, host='127.0.0.1', database='clientes')
    operation = ('; ').join(querys)
    
    try:
        
        cursor = cnx.cursor()
        result_iterator = cursor.execute(operation, multi=True)
        i = 0
        
        for res in result_iterator:
            print("Running query: ", res)
            print(f"Affected {res.rowcount} rows" )
            i += 1
            if i == len(querys): # evitar RuntimeError: 
                break # generator raised StopIteration
        cnx.commit()
        cnx.close()
        return True
    
    except:
        cnx.close()
        return "algo anda mal"
    
def validacion(a, b):
    return a == hashlib.sha1(b.lower().encode('utf-8')).hexdigest()

In [13]:
user = 'sistelca'
repo_name = 'desechosSolidos'
path_to_file = 'data_actzl.json'

finalJson = leeactzl(user, repo_name, path_to_file)


In [14]:
instrucions =  {'update': ['set', 'where'],
                 'insert into': ['(', ') values'],
                 'delete from': ['where', 'todo']
               } 

for registro in finalJson:

    dt_query = json.loads(registro['instruccion'])
    fireg, querys = calcquerys(dt_query)
    print('\n', validacion(registro['firma'], fireg))
    
    for q in querys:
        print(q)

# tentativo para luego
    if validacion(registro['firma'], fireg): # agregar condicion no se 
                                             # encuentra firma en Actzl local
            
        # agregar a querys, el query de Insert into Actzl bla bla bla
        print(conexion(querys))


 True
UPDATE datos_red SET fech_pag='2021-04-25' WHERE coduser='293'
INSERT INTO histori_pags (coduser, fech_pag, cant_Bf, fech_venc, anfit, cod) VALUES (293, '2021-05-10', 100000, '2021-05-26', '192.168.45.222', '962ad89f380951874518b9f027ef2a6f')
UPDATE datos_per SET fech_ing='2021-05-10', fech_ven='2021-05-26' WHERE coduser='293'

 False
UPDATE datos_red SETWHERE
INSERT INTO histori_pags () VALUES
UPDATE datos_per SETWHERE

 True
UPDATE datos_red SET fech_pag='2021-04-28' WHERE coduser='415'
INSERT INTO histori_pags (coduser, fech_pag, cant_Bf, fech_venc, anfit, cod) VALUES (415, '2021-05-13', 100000, '2021-05-29', '192.168.45.222', '')
UPDATE datos_per SET fech_ing='2021-05-13', fech_ven='2021-05-29' WHERE coduser='415'

 True
UPDATE datos_red SET fech_pag='2021-05-13' WHERE coduser='259'
INSERT INTO histori_pags (coduser, fech_pag, cant_Bf, fech_venc, anfit, cod) VALUES (259, '2021-05-13', 200000, '2021-06-13', '192.168.45.224', '')
UPDATE datos_per SET fech_ing='2021-05-13', fec

In [5]:
load_dotenv()

sql_id = os.getenv("SQL_ID")
sql_pw = os.getenv("SQL_PW")

for q in querys:
    print(q)


UPDATE datos_red set fech_pag='2021-05-15' where coduser='271'
INSERT INTO histori_pags (coduser, fech_pag, cant_Bf, fech_venc, anfit, cod) values (271, '2021-05-15', 200000, '2021-06-15', '192.168.45.222', '')
UPDATE datos_per set fech_ing='2021-05-15', fech_ven='2021-06-15' where coduser='271'


In [6]:
querys = ["UPDATE datos_red set fech_pag='2021-05-15' where coduser='271'",
 "INSERT INTO histori_pags (coduser, fech_pag, cant_Bf, fech_venc, anfit, cod) values (271, '2021-05-15', 200000, '2021-06-15', '192.168.45.222', '1')",
 "UPDATE datos_per set fech_ing='2021-05-15', fech_ven='2021-06-15' where coduser='271'"]

In [7]:
print(conexion(querys))

Running query:  MySQLCursor: UPDATE datos_red set fech_pag='2021-05-1..
Affected 2 rows
Running query:  MySQLCursor: INSERT INTO histori_pags (coduser, fech_..
Affected 1 rows
Running query:  MySQLCursor: UPDATE datos_per set fech_ing='2021-05-1..
Affected 1 rows
True


In [15]:
finalJson

[{'id': 1,
  'fecha': '2021-05-10 11:36:18',
  'instruccion': '{"datos_red":{"op":"UPDATE","set":" fech_pag=\'2021-04-25\' ","filtro":" coduser=\'293\'"},"histori_pags":{"op":"INSERT INTO","set":"coduser, fech_pag, cant_Bf, fech_venc, anfit, cod","filtro":" (293, \'2021-05-10\', 100000, \'2021-05-26\', \'192.168.45.222\', \'962ad89f380951874518b9f027ef2a6f\')"},"datos_per":{"op":"UPDATE","set":" fech_ing=\'2021-05-10\', fech_ven=\'2021-05-26\' ","filtro":" coduser=\'293\'"}}',
  'firma': '811fbec072f1fb1f1aeeb0dad6f00d2246a37734'},
 {'id': 2,
  'fecha': '2021-05-12 10:39:26',
  'instruccion': '{"datos_red":{"op":"UPDATE","set":"","filtro":""},"histori_pags":{"op":"INSERT INTO","set":"","filtro":""},"datos_per":{"op":"UPDATE","set":"","filtro":""}}',
  'firma': 'f89bd86bf1d39cc48b18e963e3b2127d9f4e363a'},
 {'id': 3,
  'fecha': '2021-05-13 11:16:35',
  'instruccion': '{"datos_red":{"op":"UPDATE","set":" fech_pag=\'2021-04-28\' ","filtro":" coduser=\'415\'"},"histori_pags":{"op":"INSERT I